Install pre-requisites and intialize data, model.

In [ ]:
#Install openfl and required packages for the workflow APIs to function
%pip install git+https://github.com/securefederatedai/openfl.git
%pip install -r workflow_interface_requirements.txt

#Install Tensorflow and MNIST dataset if not installed
%pip install tensorflow==2.13

# Uncomment this if running in Google Colab and set USERNAME if running in docker container.
# !pip install -r https://raw.githubusercontent.com/intel/openfl/develop/openfl-tutorials/experimental/workflow_interface_requirements.txt
# import os
# os.environ["USERNAME"] = "colab"

In [ ]:
import tensorflow as tf
import tensorflow.python.keras as keras
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import to_categorical
from keras.datasets import mnist

nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

# It is important to make sure that all values are scaled to the range [0..1] before you 
# pass them to a neural network - it is the usual convention for data preparation, 
# and all default weight initializations in neural networks are designed to work with this range.
# To achieve this:
# - Covert the integer values [0...255] to float32 
# - Divide each pixel value by 255 to get values in the range [0...1]

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.0
X_test /= 255.0

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

At this point, we have installed the necessary pre-requisites, imported required packages and downloaded the dataset.
Next we define the NN model, pre-process the data for learning and define helper functions for training.

In [ ]:
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np

# Because the output of a fully-connected layer is not normalized to be between 0 and 1, it cannot be thought of as probability. 
# Moreover, if want outputs to be probabilities of different digits, they all need to add up to 1. 
# To turn output vectors into probability vector, a function called Softmax is often used 
# as the last activation function in a classification neural network. 
# For example, softmax([−1,1,2])=[0.035,0.25,0.705].
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    MaxPool2D(), 
    Flatten(), # Converts the multi-dimensional feature map into a one-dimensional vector.
    Dense(512, activation="relu"), # A fully connected layer with 512 neurons, used to learn higher-level abstract features and representations.
    Dropout(0.2), # Randomly sets 20% of the neurons' outputs to zero during training, avoids relying on a specific neuron.
    Dense(512, activation="relu"), # A fully connected layer with 512 neurons, used to learn higher-level abstract features and representations.
    Dropout(0.2), # Randomly sets 20% of the neurons' outputs to zero during training, avoids overfitting.
    Dense(nb_classes, activation="softmax"), # To turn output vectors into probability vector,
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())


def FedAvg(models):
    new_model = models[0]
    state_dicts = [model.weights for model in models]
    state_dict = new_model.weights
    for idx, _ in enumerate(models[1].weights):
        state_dict[idx] = np.sum(np.array([state[idx]
                                 for state in state_dicts], dtype=object), axis=0) / len(models)
    new_model.set_weights(state_dict)
    return new_model

def inference(model, test_loader, batch_size):
    x_test, y_test = test_loader
    loss, accuracy = model.evaluate(
        x_test,
        y_test,
        batch_size=batch_size,
        verbose=0
    )
    accuracy_percentage = accuracy * 100
    print(f"Test set: Avg. loss: {loss}, Accuracy: {accuracy_percentage:.2f}%")
    return accuracy

Initialize the aggregator and collaborators. Edit collaborator_names to add/remove collaborators.

In [ ]:
from openfl.experimental.interface import FLSpec, Aggregator, Collaborator
from openfl.experimental.runtime import LocalRuntime
from openfl.experimental.placement import aggregator, collaborator

agg = Aggregator()

collaborator_names = ["C1", "C2"]

def callable_to_initialize_collaborator_private_attributes(n_collaborators, index, train_dataset, test_dataset, batch_size):
    from openfl.utilities.data_splitters import EqualNumPyDataSplitter
    train_splitter = EqualNumPyDataSplitter()
    test_splitter = EqualNumPyDataSplitter()

    X_train, y_train = train_dataset
    X_test, y_test = test_dataset

    train_idx = train_splitter.split(y_train, n_collaborators)
    valid_idx = test_splitter.split(y_test, n_collaborators)

    train_dataset = X_train[train_idx[index]], y_train[train_idx[index]]
    test_dataset = X_test[valid_idx[index]], y_test[valid_idx[index]]

    return {
        "train_loader": train_dataset, "test_loader": test_dataset,
        "batch_size": batch_size
    }

# Setup collaborators private attributes via callable function
collaborators = []
for idx, collaborator_name in enumerate(collaborator_names):
    collaborators.append(
        Collaborator(
            name=collaborator_name,
            num_cpus=1,
            num_gpus=0,
            private_attributes_callable=callable_to_initialize_collaborator_private_attributes,
            n_collaborators=len(collaborator_names),
            index=idx,
            train_dataset=(X_train, Y_train),
            test_dataset=(X_test, Y_test),
            batch_size=64
        )
    )

local_runtime = LocalRuntime(aggregator=agg, collaborators=collaborators, backend="ray")
print(f'Local runtime collaborators = {local_runtime.collaborators}')

Define the workflow needed to train the model using the data and participants.

In [ ]:
class KerasMNISTWorkflow(FLSpec):
    def __init__(self, model, rounds=3, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        self.n_rounds = rounds
        self.current_round = 1

    @aggregator
    def start(self):
        self.collaborators = self.runtime.collaborators
        self.next(self.aggregated_model_validation, foreach='collaborators')

    @collaborator
    def aggregated_model_validation(self):
        print(f'Performing aggregated model validation for collaborator {self.input}')
        self.agg_validation_score = inference(self.model, self.test_loader, self.batch_size)
        print(f'{self.input} value of {self.agg_validation_score}')
        self.next(self.train)

    @collaborator
    def train(self):
        x_train, y_train = self.train_loader
        history = self.model.fit(
            x_train, y_train,
            batch_size=self.batch_size,
            epochs=1,
            verbose=1,
        )
        self.loss = history.history["loss"][0]
        self.next(self.local_model_validation)

    @collaborator
    def local_model_validation(self):
        self.local_validation_score = inference(self.model, self.test_loader, self.batch_size)
        print(
            f'Doing local model validation for collaborator {self.input}: {self.local_validation_score}')
        self.next(self.join)

    @aggregator
    def join(self, inputs):
        self.average_loss = sum(input.loss for input in inputs) / len(inputs)
        self.aggregated_model_accuracy = sum(
            input.agg_validation_score for input in inputs) / len(inputs)
        self.local_model_accuracy = sum(
            input.local_validation_score for input in inputs) / len(inputs)
        print(f'Average aggregated model validation values = {self.aggregated_model_accuracy}')
        print(f'Average training loss = {self.average_loss}')
        print(f'Average local model validation values = {self.local_model_accuracy}')
        print("Taking FedAvg of models of all collaborators")
        self.model = FedAvg([input.model for input in inputs])

        self.next(self.internal_loop)

    @aggregator
    def internal_loop(self):
        if self.current_round == self.n_rounds:
            self.next(self.end)
        else:
            self.current_round += 1
            self.next(self.aggregated_model_validation, foreach='collaborators')

    @aggregator
    def end(self):
        print("Reached the end of the training flow; the model is ready to use!")

At this point we are ready to train the model with the dataset downloaded from MNIST. Call KerasMNISTWorkflow to train the model.

In [ ]:
flflow = KerasMNISTWorkflow(model, rounds=3, checkpoint=True)
flflow.runtime = local_runtime
flflow.run()